In [2]:
# Conexiones e importaciones
import os
from dotenv import load_dotenv
load_dotenv()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
import time
from urllib.parse import quote

# Spotify autentificación
mis_credenciales = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")
)

sp = spotipy.Spotify(auth_manager=mis_credenciales)

# Last.fm autentificación
api_key_lastfm = os.getenv("API_KEY_LASTFM")
shared_secret_lastfm = os.getenv("SHARED_SECRET_LASTFM")


In [3]:
generos = sp.search(q="genre:", type='track', limit=50, offset=0)

In [4]:
generos = ["flamenco", "latin", "jazz", "rock"]
año = 2018
todas_las_canciones = []
albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).
datos_artistas = {}
datos_artistas = {} 
artistas_procesados = set()

print("Buscando canciones del año", año)

for genero in generos:
    print("Género:", genero)

    # 👉 ANOTACIÓN: "genre:" SOLO funciona al buscar artistas (type="artist")
    #    No funciona con álbumes ni canciones directamente.
    resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=50)
    artistas = resultado_artistas["artists"]["items"]

    for artista in artistas:
        nombre_artista = artista["name"]

        # 👉 ANOTACIÓN: "artist:" y "year:" SÍ se pueden usar juntos
        #    al buscar álbumes (type="album") o canciones (type="track").
        #    Aquí buscamos álbumes del artista en el año indicado.
        busqueda = "artist:" + nombre_artista + " year:" + str(año)
        resultado_albumes = spotify.search(q=busqueda, type="album", limit=50)
        albumes = resultado_albumes["albums"]["items"]
        



        for album in albumes:
            id_album = album["id"]
            nombre_album = album["name"]
            
            # Si ya vimos este álbum, saltamos al siguiente
            if id_album in albumes_ya_vistos:
                continue
            
            # Si no lo habíamos visto, lo marcamos como visto
            albumes_ya_vistos.add(id_album)

            # Pedimos las canciones del álbum (esto no usa anotaciones, es una llamada directa)
            resultado_canciones = spotify.album_tracks(id_album)
            canciones = resultado_canciones["items"]

            for cancion in canciones:
                info = {
                    "nombre": cancion["name"],
                    "artista": nombre_artista,
                    "album": nombre_album,
                    "genero": genero,
                    "año": año
                }
                todas_las_canciones.append(info)

    print("  → Canciones de este género añadidas")
    print()

# Mostramos un resumen de lo encontrado
print("RESUMEN:")

# Contamos las canciones de cada género una por una
for genero in generos:
    contador = 0
    for cancion in todas_las_canciones:
        if cancion["genero"] == genero:
            contador = contador + 1
    print(f"- {genero}: {contador} canciones")

print("Total de canciones encontradas:", len(todas_las_canciones))

Buscando canciones del año 2018
Género: flamenco


NameError: name 'spotify' is not defined

In [ ]:
# PASAR LA LISTA DE DATOS A EXTRAIDOS A UN DATAFRAME CON PANDAS
todas_las_canciones_df = pd.DataFrame(todas_las_canciones)
todas_las_canciones_df

,nombre,artista,album,genero,año
0,NI BIEN NI MAL,Bad Bunny,X 100PRE,latin,2018
1,200 Mph,Bad Bunny,X 100PRE,latin,2018
2,¿Quien Tu Eres?,Bad Bunny,X 100PRE,latin,2018
3,Caro,Bad Bunny,X 100PRE,latin,2018
4,Tenemos Que Hablar,Bad Bunny,X 100PRE,latin,2018
...,...,...,...,...,...
9079,Dulce Noche,Maná,Buenas Vibraciones - Música para Limpiar la En...,rock,2018
9080,Calmante Piano Music,Maná,Buenas Vibraciones - Música para Limpiar la En...,rock,2018
9081,Salón Oriental,Maná,Buenas Vibraciones - Música para Limpiar la En...,rock,2018
9082,Increíble,Maná,Buenas Vibraciones - Música para Limpiar la En...,rock,2018


In [ ]:
# PASAR DATAFRAME A UN ARCHIVO CSV
todas_las_canciones_df.to_csv("canciones_2018.csv", index=False)

In [ ]:
# LAST FM

if not api_key_lastfm:
    print("ERROR: La variable de entorno 'API_KEY_LASTFM' no está configurada.")
else:
    print("API Key de Last.fm cargada con éxito.")

API Key de Last.fm cargada con éxito.


In [ ]:
url_last_fm = ("http://ws.audioscrobbler.com/2.0/")

In [ ]:
# EXTRAER INFO DE ARTISTAS DE API LASTFM
def busqueda_info_artista(nombre_artista, api_key_lastfm):
    artista_codificado = quote(nombre_artista) #esto sirve para que los espacios y caracteres especiales no interfieran con la url de la api
    
    url_last_fm = "http://ws.audioscrobbler.com/2.0/"
    params_info = {
                        'method': 'artist.getinfo',
                        'artist': artista_codificado,
                        'api_key': api_key_lastfm,
                        'format': 'json'
                    }
    try:
        response = requests.get(url_last_fm, params=params_info, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "artist" in data: 
            artista_info = data['artist']
            tags = [tag['name'] for tag in artista_info.get('tags', {}).get('tag', [])]
            bio_summary = artista_info.get('bio', {}).get('summary', '').split('<a href')[0].strip()
            return {
                        'tags_lastfm': ", ".join(tags),
                        'bio_resumen': bio_summary,
                        'listeners': int(artista_info.get('stats', {}).get('listeners', 0)),
                        'consulta_exitosa': True
        }
        else:
            # Artista no encontrado por Last.fm
            return {'consulta_exitosa': False, 'error_lastfm': "No encontrado en Last.fm"}
        
    except requests.exceptions.RequestException as e:
        # Incluye HTTPError, ConnectionError, TimeoutError, etc.
        status_code = getattr(e.response, 'status_code', 'N/A')
        return {'consulta_exitosa': False, 'error_lastfm': f"Error API ({status_code}): {e}"}
    except Exception as e:
        return {'consulta_exitosa': False, 'error_lastfm': f"Error Procesamiento: {e}"}


if not api_key_lastfm:
    print("ERROR: La clave de la API de Last.fm no está configurada.")
else:
    artistas_unicos = todas_las_canciones_df['artista'].unique() #extrae los artistas sin repetir
    print(f"\nTotal de artistas únicos a consultar en Last.fm: {len(artistas_unicos)}")

    artistas_df = pd.DataFrame(artistas_unicos, columns=['artista']) #df temporal con los artistas sin repetir
    
    print("\nIniciando consultas a Last.fm...")
    
    #apply pasa el valor de la columna "artista" como primer argumento posicional. Con "args" se pasan el resto de argumentos.
    resultados_lastfm_serie = artistas_df['artista'].apply(
        busqueda_info_artista, 
        args=(api_key_lastfm,) 
    )

    # 4. Normalizar los resultados (convertir la Serie de Diccionarios a Columnas de DF)
    datos_lastfm_df = pd.json_normalize(resultados_lastfm_serie)
    datos_lastfm_df.insert(0, 'artista', artistas_unicos)

    # 6. Unir los datos de Last.fm al DataFrame principal de canciones
    df_final = pd.merge(
        todas_las_canciones_df,
        datos_lastfm_df, # ¡Usamos el DF recién creado!
        on='artista',
        how='left'
    )
    
    print("Consultas a Last.fm terminadas y datos unidos al DataFrame.")


Total de artistas únicos a consultar en Last.fm: 121

Iniciando consultas a Last.fm...
Consultas a Last.fm terminadas y datos unidos al DataFrame.


In [ ]:
datos_lastfm_df

,artista,tags_lastfm,bio_resumen,listeners,consulta_exitosa
0,Bad Bunny,"Reggaeton, latin, trap, puerto rico, trap latino","Benito Antonio Martínez Ocasio (born March 10,...",2117387,True
1,Morat,"colombian, latin pop, folk pop, latino, pop",Morat is a Colombian band formed in the countr...,291934,True
2,ROSALÍA,"pop, Flamenco, spanish, Flamenco Nuevo, spain","Rosalia Vila Tobella, known mononymously as RO...",1734793,True
3,Kapo,"acoustic, Colombia",The band Kapo was formed in 2006 by Joe Muller...,127043,True
4,Duki,"trap, latin, Latin Trap, argentina, rap","Mauro Ezequiel Lombardo Quiroga, conocido artí...",283091,True
...,...,...,...,...,...
116,Imagine Dragons,"indie, indie rock, alternative, rock, indie pop",Imagine Dragons is an American pop band formed...,3943396,True
117,Maroon 5,"pop rock, rock, pop, alternative, alternative ...",Maroon 5 is an American pop rock band that ori...,6434168,True
118,Elton John,"pop, classic rock, singer-songwriter, rock, piano",Sir Elton Hercules John (born Reginald Kenneth...,5072416,True
119,Juanes,"latin, pop, spanish, rock, latino",Juan Esteban Aristizábal Vásquez (born August ...,1170421,True


In [ ]:
datos_lastfm_df
datos_lastfm_df.to_csv("datos_lastfm_2018.csv", index=False)

In [ ]:
# CREACION DE LA BASE DE DATOS

# A. Conexión a MySQL
try:
    # Intentamos conectarnos al servidor MySQL
    cnx = mysql.connector.connect(
        host='127.0.0.1',    # Dirección del servidor MySQL (localhost)
        user='root',         # Usuario con el que nos conectamos
        password='AlumnaAdalab#',  # Contraseña del usuario
        use_pure=True,       # use_pure=True garantiza compatibilidad con Python 3.12, evita problemas en el Kernel
    )
    print('Conexión exitosa')  # Confirmamos que la conexión fue correcta
except mysql.connector.Error as e:  # Captura errores relacionados con MySQL
    print('Error al conectar:', e)   # Muestra el error para diagnosticar el problema

NameError: name 'mysql' is not defined